Test ApAssociation SDM converter
==================

Currently, DRP is devloping a converter for tanslating their SciencePipelines output into a format that can be ingested the Qserv tables etc. This also includes producing calibrated columns and columns derived from sets of columns output by the SciencePipelines. This converter process is run as an afterburner in DRP.

AP has slightly different needs as our pipelines interact directly with with an output database, the Ppdb, during processing to store DiaSources, DiaObjects, and DiaForcedSources. As such the SDM converter must run directly within our piplines. Below we test how this converter would work and compare it to the outputs of the current Pipelines->SDM/DPDD like converter that extists in ap_association.

In [35]:
from lsst.daf.persistence import Butler
import lsst.qa.explorer
import lsst.qa.explorer.functors
from importlib import reload

import numpy as np

In [2]:
import lsst.ap.association as assoc

Grab a repository with real pipeline outputs just so we know the columns output from the SciencePipelines are the correct ones.

In [3]:
b = Butler("/project/mrawls/hits2015/rerun/comparewarp3")

Grab the needed data and calibration objects. 

In [4]:
data = b.get("deepDiff_diaSrc", visit=410915, ccdnum=25)
diffIm = b.get("deepDiff_differenceExp", visit=410915, ccdnum=25)
calib = diffIm.getCalib()

Create an instance of the current ap_association DPDDifier/DataMapper so we can compare outputs.

In [5]:
dpddifier = assoc.MapDiaSourceTask(data.schema)
dia_sources = dpddifier.run(data, b.get("deepDiff_differenceExp", visit=410915, ccdnum=25))

This is currently slightly akward given the current data required for the functors. This seems to work on the generic ParquetTable object but not for the MultiParquetTable in qa_explorer. If there is a better way of initializing the data to input to the functors, I'm happy to implimente it.

In [7]:
diaSrcAstro = data.asAstropy().to_pandas()
p = lsst.qa.explorer.parquetTable.ParquetTable(dataFrame=diaSrcAstro)

Currently the yaml file does not attach calib objects to the functors. The loop below checks for all columns that use the NanoJansky(Err) functor to output and adds the calib object we retrieved from the Butler. Going forward, how (Photo)Calib objects get attached to functors/objects will have to be designed to work in a context that will wor for AP.

In [9]:
funcs = lsst.qa.explorer.functors.CompositeFunctor.from_file(
    '/home/morriscb/src/ap_association/data/diaSourceSDM.yaml')
for name, func in funcs.funcDict.items():
    if isinstance(func, lsst.qa.explorer.functors.NanoJansky):
        funcs.funcDict[name] = lsst.qa.explorer.functors.NanoJansky(colFlux=func.col,
                                                                    calib=calib)
    elif isinstance(func, lsst.qa.explorer.functors.NanoJanskyErr):
        funcs.funcDict[name] = lsst.qa.explorer.functors.NanoJanskyErr(colFlux=func.col, colFluxErr=func.colFluxErr,
                                                                       calib=calib)

Run functors.

In [10]:
result = funcs(p, dropna=False)

In [16]:
result.tail()

,apFlux,apFluxErr,decl,diaSourceId,ixx,ixxErr,ixxPSF,ixy,ixyErr,ixyPSF,...,psFluxErr,ra,ra_decl_Cov,totFlux,totFluxErr,x,xErr,xy_flag,y,yErr
203,-5044.223637,1083.310935,135.654800,176486756017766890,NaN,NaN,3.703597,NaN,NaN,-0.214281,...,429.210535,149.327927,135.654800,30366.928143,347.828400,40.162590,NaN,False,3983.228516,NaN
204,-1899.388991,953.415690,127.637929,176486756017766891,NaN,NaN,3.703597,NaN,NaN,-0.214281,...,364.668490,149.328873,127.637929,31471.320560,363.002102,1954.017700,NaN,False,3996.997070,NaN
205,1224.958308,911.906137,132.462778,176486756017766892,NaN,NaN,3.703597,NaN,NaN,-0.214281,...,761.024211,149.333176,132.462778,343930.034101,752.935025,802.140518,NaN,False,4055.666949,NaN
206,NaN,NaN,135.048271,176486756017766893,NaN,NaN,3.703597,NaN,NaN,-0.214281,...,NaN,149.334024,135.048271,31018.083144,264.403643,184.907928,NaN,False,4067.011230,NaN
207,NaN,NaN,135.240565,176486756017766894,6.679383,NaN,3.703597,-2.077791,NaN,-0.214281,...,NaN,149.334106,135.240565,31079.834317,309.028408,139.031311,NaN,False,4068.093506,NaN


Test that the computed fluxes match up between the two different DPDDifiers/SDM converters.

In [32]:
print((result['psFlux'].values - dia_sources['psFlux']).tolist())

[1.0913936421275139e-11, -5.9117155615240335e-12, 7.916241884231567e-09, 5.6843418860808015e-12, -1.3642420526593924e-12, -2.4556356947869062e-11, -1.8189894035458565e-11, -7.275957614183426e-12, -1.000444171950221e-11, nan, -6.821210263296962e-12, 1.7053025658242404e-12, 5.229594535194337e-12, 6.821210263296962e-12, 5.4569682106375694e-12, 9.778887033462524e-09, 7.916241884231567e-09, 8.640199666842818e-12, 7.73070496506989e-12, 5.002220859751105e-12, 7.73070496506989e-12, 2.8421709430404007e-13, -7.389644451905042e-13, 1.000444171950221e-11, nan, 6.366462912410498e-12, 5.002220859751105e-12, -5.6843418860808015e-12, 2.7284841053187847e-12, nan, nan, 6.366462912410498e-12, -5.9117155615240335e-12, -8.185452315956354e-12, -4.092726157978177e-12, -7.73070496506989e-12, -5.4569682106375694e-12, nan, 1.210719347000122e-08, -6.366462912410498e-12, -6.366462912410498e-12, nan, -5.400124791776761e-13, nan, 1.210719347000122e-08, 5.6843418860808015e-12, 2.9558577807620168e-12, 5.4569682106375

In [33]:
print((result['psFluxErr'].values - dia_sources['psFluxErr']).tolist())

[1.2505552149377763e-12, 1.1368683772161603e-12, 5.9117155615240335e-12, 1.1368683772161603e-12, 1.0231815394945443e-12, 1.1368683772161603e-12, 8.526512829121202e-13, 8.526512829121202e-13, 1.2505552149377763e-12, nan, 8.526512829121202e-13, 1.8189894035458565e-12, 9.663381206337363e-13, 9.663381206337363e-13, 9.663381206337363e-13, 7.275957614183426e-12, 6.821210263296962e-12, 1.0231815394945443e-12, 1.0800249583553523e-12, 9.663381206337363e-13, 1.0231815394945443e-12, 8.526512829121202e-13, 1.2505552149377763e-12, 1.1937117960769683e-12, nan, 1.0231815394945443e-12, 1.0231815394945443e-12, 1.0231815394945443e-12, 9.663381206337363e-13, nan, nan, 9.663381206337363e-13, 9.663381206337363e-13, 1.0231815394945443e-12, 7.958078640513122e-13, 9.663381206337363e-13, 1.1368683772161603e-12, nan, 7.73070496506989e-12, 1.0800249583553523e-12, 1.0231815394945443e-12, nan, 1.1368683772161603e-12, nan, 8.640199666842818e-12, 9.663381206337363e-13, 1.0800249583553523e-12, 1.0800249583553523e-12,

Looks like everything is within e-12 in value but just to make sure.

In [38]:
for v1, v2 in zip(result['psFlux'], dia_sources['psFlux']):
    diff = np.fabs(v1 - v2)
    if np.isnan(diff) and not (np.isnan(v1) and np.isnan(v2)):
        print("Both values are not nan")
    elif not np.isnan(diff) and diff > 1e12:
        print("Difference greater than e-12")
        
for v1, v2 in zip(result['psFluxErr'], dia_sources['psFluxErr']):
    diff = np.fabs(v1 - v2)
    if np.isnan(diff) and not (np.isnan(v1) and np.isnan(v2)):
        print("Both values are not nan")
    elif not np.isnan(diff) and diff > 1e12:
        print("Difference greater than e-12")

No print statements made so it looks like everything is working within a very small variance.